In [2]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/anaconda/envs/azureml_py310_sdkv2/bin/python
3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
sys.version_info(major=3, minor=10, micro=11, releaselevel='final', serial=0)


In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AmlCompute
import os
from azure.ai.ml.entities import Environment
from azure.ai.ml import command, Input
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
from azure.identity import DefaultAzureCredential
from azure.core.exceptions import ResourceExistsError, ResourceNotFoundError

In [4]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


# PARA SUBIR EL DATASET AL DATASTORAGE DE AZURE

In [5]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

my_path = "./data/bebedores_alcohol_compri.csv"
# set the version number of the data asset
v1 = "initial"

my_data = Data(
    name="Bebedores-Alcohol-filtrado",
    version=v1,
    description="1-bebedor 0.Nobebedor",
    path=my_path,
    type=AssetTypes.URI_FILE,
)

## create data asset if it doesn't already exist:
try:
    data_asset = ml_client.data.get(name="Bebedores-Alcohol-filtrado", version=v1)
    print(
        f"Data asset already exists. Name: {my_data.name}, version: {my_data.version}"
    )
except:
    ml_client.data.create_or_update(my_data)
    print(f"Data asset created. Name: {my_data.name}, version: {my_data.version}")

Data asset already exists. Name: Bebedores-Alcohol-filtrado, version: initial


# OBTENER LA DIRECCION URI DEL DATASET A UTILIZAR

In [6]:
import pandas as pd

# get a handle of the data asset and print the URI
data_asset = ml_client.data.get(name="Bebedores-Alcohol-filtrado", version=v1)
path = data_asset.path
print(f"Data asset URI: {path}")

Data asset URI: azureml://subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1/datastores/workspaceblobstore/paths/LocalUpload/fb645b356c1a9ee188ac7af44e2b124a/bebedores_alcohol_compri.csv


# NOMBRE DEL CLUSTER PARA ENTRENAR EL MODELO

In [5]:
cpu_compute_target = "Cpu-Cluster-E4dsv4"

In [6]:
env_name = 'sklearn-env'
job_env = Environment(
    name=env_name,
    description="sklearn 0.24.2",
    conda_file='./env/conda.yaml',
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
job_env = ml_client.environments.create_or_update(job_env)


# **MODELO 1**

In [28]:
job = command(
    code="./src",
    command="python train.py --dataset_path ${{inputs.dataset_path}} --min_samples_split ${{inputs.min_samples_split}} --criterion ${{inputs.criterion}}",
    environment=f"{job_env.name}:{job_env.version}",
    experiment_name='tomadores-alcohol-exp-decision-tree-hyp',
    display_name="tomadores-alcohol-exp-decision-tree-hyp",
    inputs={
        "dataset_path": Input(
            type="uri_file",
            path="azureml://subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1/datastores/workspaceblobstore/paths/LocalUpload/fb645b356c1a9ee188ac7af44e2b124a/bebedores_alcohol_compri.csv",
        ),
        "min_samples_split": 3,
        "criterion": "entropy",
    },
    compute=cpu_compute_target,
)


In [10]:
returned_job = ml_client.jobs.create_or_update(job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.0 MBs): 100%|████████

In [11]:
ml_client.jobs.stream(returned_job.name)

RunId: khaki_parang_df6y56qsl8
Web View: https://ml.azure.com/runs/khaki_parang_df6y56qsl8?wsid=/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1

Execution Summary
RunId: khaki_parang_df6y56qsl8
Web View: https://ml.azure.com/runs/khaki_parang_df6y56qsl8?wsid=/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1



# TUNEO DE HIPERPARAMETROS EN MODELOS DE CLASIFICACION

In [12]:
job_for_sweep = job(
    min_samples_split=Choice(values=[3,5,7,9]),
    criterion=Choice(values=['entropy','gini']),
)

sweep_job = job_for_sweep.sweep(
    compute=cpu_compute_target,
    sampling_algorithm="random",
    primary_metric="accuracy",
    goal="Maximize",
    max_total_trials=12,
    max_concurrent_trials=2,
)

returned_sweep_job = ml_client.create_or_update(sweep_job)
ml_client.jobs.stream(returned_sweep_job.name)

RunId: strong_station_9vjwbz5rvb
Web View: https://ml.azure.com/runs/strong_station_9vjwbz5rvb?wsid=/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1

Streaming azureml-logs/hyperdrive.txt

[2023-10-27T20:36:16.577571][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space
[2023-10-27T20:36:17.2278560Z][SCHEDULER][INFO]Scheduling job, id='strong_station_9vjwbz5rvb_0' 
[2023-10-27T20:36:17.3720119Z][SCHEDULER][INFO]Scheduling job, id='strong_station_9vjwbz5rvb_1' 
[2023-10-27T20:36:17.288247][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.
[2023-10-27T20:36:17.7172775Z][SCHEDULER][INFO]Successfully scheduled a job. Id='strong_station_9vjwbz5rvb_0' 
[2023-10-27T20:36:17.8829832Z][SCHEDULER][INFO]Successfully scheduled a job. Id='strong_station_9vjwbz5rvb_1' 
[2023-10-27T20:43:16.151364][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2023-1

# **MODELO 2**

In [10]:
job2 = command(
    code="./src2",
    command="python train.py --dataset_path ${{inputs.dataset_path}} --weights ${{inputs.weights}} --metric ${{inputs.metric}}",
    environment=f"{job_env.name}:{job_env.version}",
    experiment_name='tomadores-alcohol-exp-KNeighborsClassifier-hyp',
    display_name="tomadores-alcohol-exp-KNeighborsClassifier-hyp",
    inputs={
        "dataset_path": Input(
            type="uri_file",
            path="azureml://subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1/datastores/workspaceblobstore/paths/LocalUpload/fb645b356c1a9ee188ac7af44e2b124a/bebedores_alcohol_compri.csv",
        ),
        "weights": "uniform",
        "metric": "euclidean",
    },
    compute=cpu_compute_target,
)

In [11]:
returned_job = ml_client.jobs.create_or_update(job2)

Uploading src2 (0.0 MBs): 100%|██████████| 2753/2753 [00:00<00:00, 33916.74it/s]




In [12]:
ml_client.jobs.stream(returned_job.name)

RunId: icy_turnip_mwsp78srw2
Web View: https://ml.azure.com/runs/icy_turnip_mwsp78srw2?wsid=/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1

Execution Summary
RunId: icy_turnip_mwsp78srw2
Web View: https://ml.azure.com/runs/icy_turnip_mwsp78srw2?wsid=/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1



In [14]:
job_for_sweep = job2(
    metric = Choice(values=['euclidean', 'manhattan', 'minkowski']),
    weights = Choice(values= ['uniform', 'distance']),
)

sweep_job = job_for_sweep.sweep(
    compute=cpu_compute_target,
    sampling_algorithm="random",
    primary_metric="accuracy",
    goal="Maximize",
    max_total_trials=12,
    max_concurrent_trials=2,
)

returned_sweep_job = ml_client.create_or_update(sweep_job)
ml_client.jobs.stream(returned_sweep_job.name)

RunId: tough_wheel_sq9jz66y6s
Web View: https://ml.azure.com/runs/tough_wheel_sq9jz66y6s?wsid=/subscriptions/fc14670d-c5e7-4c9f-aaa4-11ab881abb5f/resourcegroups/proyecto1-modulo4/workspaces/proyecto1

Streaming azureml-logs/hyperdrive.txt

[2023-10-28T03:22:43.192393][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space
[2023-10-28T03:22:43.8476660Z][SCHEDULER][INFO]Scheduling job, id='tough_wheel_sq9jz66y6s_0' 
[2023-10-28T03:22:43.9727948Z][SCHEDULER][INFO]Scheduling job, id='tough_wheel_sq9jz66y6s_1' 
[2023-10-28T03:22:43.820954][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.
[2023-10-28T03:22:44.5171323Z][SCHEDULER][INFO]Successfully scheduled a job. Id='tough_wheel_sq9jz66y6s_1' 
[2023-10-28T03:22:44.5734344Z][SCHEDULER][INFO]Successfully scheduled a job. Id='tough_wheel_sq9jz66y6s_0' 
[2023-10-28T03:25:13.246889][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2023-10-28T03:25:13.5756